In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingRegressor
import requests
from scipy import stats
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE, SelectKBest, chi2
import gc
# from tensorflow.keras.layers import *
# import tensorflow as tf


In [3]:
train_features = pd.read_csv('./X_train.csv')
train_targets = pd.read_csv('./y_train.csv')
test_data = pd.read_csv('./X_test.csv')

In [ ]:
# train_data = train_data.drop(['id', 'Moons'], axis=1)
# test_data = test_data.drop(['id', 'Moons'], axis=1)
# train_targets = train_targets.drop(['id', 'Moons'], axis=1)

In [ ]:
train_features.head()

,id,Moons,brisk_Feature_1,brisk_Feature_2,brisk_Feature_3,brisk_Feature_4,brisk_Feature_5,brisk_Feature_6,brisk_Feature_7,brisk_Feature_8,...,vratios_Feature_115,vratios_Feature_116,vratios_Feature_117,vratios_Feature_118,vratios_Feature_119,vratios_Feature_120,vratios_Feature_121,vratios_Feature_122,vratios_Feature_123,vratios_Feature_124
0,21909,0,0.67,0.83,0.67,0.50,0.83,0.83,0.50,0.17,...,0.17,0.33,0.50,0.83,0.33,0.17,0.17,0.33,0.67,0.50
1,200925,0,0.33,0.50,0.50,1.00,0.33,0.83,0.17,0.67,...,0.00,0.17,0.67,0.17,0.33,0.17,0.17,0.17,0.50,0.67
2,92292,0,0.33,0.67,0.33,0.83,0.50,0.50,0.50,0.83,...,0.17,0.17,0.67,0.67,0.50,0.33,0.17,0.17,0.83,0.67
3,4832,0,0.33,0.67,0.50,0.33,0.67,0.33,0.33,0.67,...,0.50,0.50,0.50,0.50,0.33,0.50,0.50,0.67,0.67,0.50
4,300588,0,0.67,0.33,0.33,0.67,0.83,0.17,0.33,0.50,...,0.67,0.83,0.67,0.00,0.17,0.33,0.50,0.50,0.33,0.50


In [ ]:
train_targets.head()

,id,Moons,target_w,target_r,target_g,target_b
0,21909,0,0.33,0.50,0.50,0.67
1,200925,0,0.50,0.33,0.17,0.50
2,92292,0,0.50,0.50,0.50,0.50
3,4832,0,0.50,0.67,0.67,0.83
4,300588,0,0.67,0.67,0.83,0.83


In [ ]:
test_data.head()

,id,Moons,brisk_Feature_1,brisk_Feature_2,brisk_Feature_3,brisk_Feature_4,brisk_Feature_5,brisk_Feature_6,brisk_Feature_7,brisk_Feature_8,...,vratios_Feature_115,vratios_Feature_116,vratios_Feature_117,vratios_Feature_118,vratios_Feature_119,vratios_Feature_120,vratios_Feature_121,vratios_Feature_122,vratios_Feature_123,vratios_Feature_124
0,65255,210,0.67,0.83,0.83,0.33,0.17,0.83,0.00,0.50,...,0.17,0.17,0.33,0.50,0.33,0.33,0.33,0.33,0.33,0.50
1,272814,210,0.17,0.50,0.33,1.00,0.50,0.17,0.33,0.67,...,0.17,0.33,0.67,1.00,0.50,0.50,0.33,0.33,1.00,0.67
2,144390,210,0.33,0.50,0.33,0.33,0.50,0.83,0.67,0.17,...,0.83,0.83,0.33,0.67,0.33,0.33,0.67,0.67,0.50,0.33
3,216248,210,0.83,0.83,0.33,0.17,0.83,0.50,0.67,0.50,...,0.83,0.83,0.67,0.50,0.50,0.83,0.83,0.50,0.50,0.67
4,165469,210,0.67,0.00,0.67,0.50,0.67,0.33,0.33,0.33,...,0.67,0.67,0.17,0.83,0.67,0.50,0.83,0.83,0.83,0.17


In [ ]:
# Merge train_features and train_targets for ease of use
train_data = pd.merge(train_features, train_targets, on=['id', 'Moons'], how='inner')

del train_features, train_targets
gc.collect()

105

In [ ]:
# Get the features columns name  and the targets columns name
features = [col for col in train_data.columns if 'Feature' in col]
targets = [col for col in train_data.columns if 'target' in col]

# Modeling



In [ ]:
def xg_boost_template(X_train, y_train, X_test, y_test, test_refs, target):
    model = xgb.XGBRegressor(objective='reg:squarederror', max_depth=3, learning_rate=0.01, n_estimators=400, n_jobs=-1, colsample_bytree=0.5)
    model.fit(X_train, y_train[target], verbose=True)

    # Test the spearman of your model on the X_test data
    preds = pd.DataFrame(model.predict(X_test), columns=[target])
    get_spearman_results(preds, y_test, test_refs)

    return model

In [ ]:
def get_spearman_results(preds, y_test, test_refs):
    preds.rename({f'{target}':f'pred_{target.split("_")[1]}' for target in preds.columns}, axis=1, inplace=True)
    preds_ref = pd.concat([preds.reset_index(drop=True), test_refs.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)
    spearman = pd.DataFrame()
    
    target_suffixes = [col.split('_')[-1] for col in preds.columns if 'pred' in col]
    for suffix in target_suffixes:
        spearman[f'target_{suffix}'] = preds_ref.groupby('Moons')[[f'pred_{suffix}', f'target_{suffix}']].corr(method='spearman').unstack().iloc[:,1]

    print(f'\nSpearman score over the period :\n{spearman.describe()}\n')
    return spearman

In [ ]:
def train_test_split(data):
    number_of_moons = len(data['Moons'].unique())
    embargo = 13 # Embargo between train and test set
    proportion = 0.8

    # Train on 80% of the first moons and test on 20% of the last moons
    train_set = data[data['Moons'] < int(number_of_moons * proportion) - embargo]
    test_set = data[data['Moons'] > int(number_of_moons * proportion)]

    X_train = train_set[features]
    y_train = train_set[targets]
    X_test = test_set[features]
    y_test = test_set[targets]
    test_refs = test_set.iloc[:, :2]

    return X_train, y_train, X_test, y_test, test_refs

In [ ]:
# Split your data to test your model
X_train, y_train, X_test, y_test, test_refs = train_test_split(train_data)

In [ ]:
# Run your model on the different targets
model = {}
for target in targets:
  model[f'xgb_model_{target}'] = xg_boost_template(X_train, y_train, X_test, y_test, test_refs, target)


Spearman score over the period :
        target_w
count  39.000000
mean    0.022798
std     0.123172
min    -0.252432
25%    -0.065849
50%     0.040134
75%     0.101779
max     0.271028


Spearman score over the period :
        target_r
count  39.000000
mean    0.033406
std     0.069630
min    -0.139671
25%    -0.020652
50%     0.045443
75%     0.087333
max     0.186558


Spearman score over the period :
        target_g
count  39.000000
mean    0.037326
std     0.081896
min    -0.138805
25%    -0.007020
50%     0.026069
75%     0.094360
max     0.193678


Spearman score over the period :
        target_b
count  39.000000
mean    0.020969
std     0.080223
min    -0.119055
25%    -0.031453
50%     0.008693
75%     0.078044
max     0.173927



# Features Selection

## Using RFE

In [ ]:
# def rfe_selection(model, test_data_input, train_data_input, target):

#     print('\nSelecting features...')
#     recur = RFE(estimator = model, n_features_to_select = 15)
#     features = recur.fit(train_data_input, target)

#     # print(f'Number of features: {features.n_features_} ')
#     # print(f'Selected Features are: {features.support_}')
#     # print(f'Features rangking is as follows: {features.ranking_}')
#     new_features = []

#     print('Create new features...')
#     for i, v in enumerate(features.support_):
#         if v == True:
#             new_features.append(train_data_input.columns[i])
    
#     new_train_data = train_data_input[new_features]
#     #print(new_train_data.columns)
#     new_test_data = test_data_input[new_features]
#     #print(new_test_data.columns)

#     # print('Modeling with new features...')
#     # model = xg_boost_hackathon(model, new_train_data, target)


#     return new_test_data, new_train_data



In [ ]:
# test_data_r, train_data_r = rfe_selection(model, test_data, train_data, train_targets.target_r)
# test_data_g, train_data_g= rfe_selection(model, test_data, train_data, train_targets.target_g)
# test_data_b, train_data_b= rfe_selection(model, test_data, train_data,train_targets.target_b)

In [ ]:
# test_data_r.columns

In [ ]:
# train_data_r.columns

In [ ]:
# rfe_model_r= xg_boost_hackathon(model, train_data_r, train_targets.target_r)
# rfe_model_g= xg_boost_hackathon(model, train_data_g, train_targets.target_g)
# rfe_model_b= xg_boost_hackathon(model, train_data_b, train_targets.target_b)

# Using Chi Score

In [ ]:
# chi2_selector = SelectKBest(chi2, k=50)

In [ ]:
# x_best = chi2_selector.fit_transform(train_data, train_targets.target_r.astype('int'))

In [ ]:
# x_best.shape

In [ ]:
# model_target_r_chi= xg_boost_hackathon(model, x_best, train_targets.target_r)
# model_target_g_chi= xg_boost_hackathon(model,x_best, train_targets.target_g)
# model_target_b_chi= xg_boost_hackathon(model, x_best, train_targets.target_b)

# make a prediction

In [ ]:
prediction = test_data.iloc[:, :2]
for target in targets:
    prediction.loc[:, target] = model[f'xgb_model_{target}'].predict(test_data.iloc[:, 2:])

In [ ]:
prediction

,id,Moons,target_w,target_r,target_g,target_b
0,65255,210,0.497975,0.503545,0.508089,0.519127
1,272814,210,0.494868,0.496361,0.500741,0.502093
2,144390,210,0.495749,0.483553,0.461289,0.446090
3,216248,210,0.501871,0.513632,0.518857,0.524160
4,165469,210,0.487641,0.498687,0.494626,0.490841
...,...,...,...,...,...,...
131862,35742,367,0.510983,0.527825,0.580756,0.603441
131863,241636,367,0.494817,0.490453,0.479250,0.464716
131864,290557,367,0.498094,0.499855,0.497842,0.495738
131865,104147,367,0.499656,0.499997,0.501793,0.501864


In [ ]:
# # Check the validity of your prediction dataframe

# for moon in prediction['Moons'].unique():
#     # Check if you have the same id than in the example_submission
#     if not set(prediction[prediction['Moons'] == moon]['id'].unique()) == set(exemple_submission[exemple_submission['Moons'] == moon]['id'].unique()):
#         print('Your dataframe does not contain the ids required for each moon')
#     for target in targets:
#         # Check if you have constants for each moon and each target
#         if prediction[prediction['Moons'] == moon][target].nunique() == 1:
#             print(f'Your submission contains constant values on moon : {moon}')

# # Check if your columns are well named
# if prediction.columns.tolist() != exemple_submission.columns.tolist():
#     print('Your columns name does not match the exemple_submission columns')

# # Check if your values are between 0 and 1
# if (prediction.iloc[:, 2:].values > 1).any() or (prediction.iloc[:, 2:].values < 0).any():
#     print('At least one value is higher than 1 or lower than 0')

In [ ]:
API_KEY = "" # <- enter your API key HERE

r = requests.post("https://tournament.crunchdao.com/api/v2/submissions",
    files = {
        "file": ("x", prediction.to_csv(index=False).encode('ascii'))
    },
    data = {
        "apiKey": API_KEY, 
        "comment": 'My xgboost model', # <- Enter your comment here
        # "model": '', # Coming soon...
    },
)

if r.status_code == 200:
    print("Submission submitted.")
elif r.status_code == 423:
    print("ERR: Submissions are close")
    print("You can only submit during rounds eg: Friday 7pm GMT+1 to Sunday midnight GMT+1.")
    print("Or the server is currently crunching the submitted files, please wait some time before retrying.")
elif r.status_code == 422:
    print("ERR: API Key is missing or empty")
    print("Did you forget to fill the API_KEY variable?")
elif r.status_code == 400:
    print("ERR: The file must not be empty")
    print("You have send a empty file.")
elif r.status_code == 401:
    print("ERR: Your email hasn't been verified")
    print("Please verify your email or contact a cruncher.")
elif r.status_code == 409:
    print("ERR: Duplicate submission")
    print("Your work has already been submitted with the same exact results, if you think that this a false positive, contact a cruncher.")
    print("MD5 collision probability: 1/2^128 (source: https://stackoverflow.com/a/288519/7292958)")
elif r.status_code == 429:
    print("ERR: Too many submissions")
else:
    print("ERR: Server returned: " + str(r.status_code))
    print("Ouch! It seems that we were not expecting this kind of result from the server, if the probleme persist, contact a cruncher.")

Submission submitted.
